<a href="https://colab.research.google.com/github/MinatoIsuki/AI-Song-Cover-RVC/blob/main/Training_V2_and_Youtube_Audio_Download_%26_Splitting_Audio_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **RVC Training and Youtube Audio Download & Splitting Audio - All in One**

---


RVC Training V2 and Youtube Audio Download & Splitting Audio script by Ardha<br>
Combined by Minato Isuki

<br>*If there is any issue within the code (or any typo), please reply to my pull request or create a issue then mention my pull request.*

**Please input all information in these two input form before executing "Run All".**<br>
**If your audio is in Google Drive, please execute the mount Google Drive script at Step 2.1 before filling the form.**

#### **How to get Google Drive path**
*Click the arrow to show instructions*

[Video instructions](https://img.guildedcdn.com/ContentMediaGenericFiles/f380a54fb1065b2f12dad7b4ba09211e-Full.mp4?w=1280&h=720)<br>
*Image instructions will be created (or not, Colab uses base64 which lags my PC pretty bad).*

### **Step 1: Filling form (For audio download and for RVC)**

In [ ]:
#@title Download Youtube Audio - Input Form

#@markdown If you are making a new model (or getting new audio for a model), use Splitting mode, else, use Seperate mode for music seperation.
AudioMode = "Separate" #@param ["Separate", "Splitting"]
#@markdown Where did you store your audio/music at?
dataset = "Youtube" #@param ["Youtube", "Drive"]
#@markdown If it's YouTube, enter the URL here.
url = "https://youtu.be/e99Rq5pW6vU" #@param {type:"string"}
#@markdown If it's Google Drive, start the mount Google Drive script in Step 2.1 then enter the path here. The path should begin with /content/drive/MyDrive.
drive_path = "" #@param {type:"string"}
#@markdown Enter the music name or the sound model here.
AUDIO_NAME = "TheLostSoulDown" #@param {type:"string"}


In [ ]:
#@title RVC Training - Input Form
#@markdown Select "inference" to mess with your AI model (maybe making an AI cover).<br>
#@markdown Select "training" if you are going to make a new model or continue training.
RVCMode = "inference" #@param ["inference", "training"]
#@markdown Enter your AI model name to inference/continue training
MODELNAME = "kanna"  #@param {type:"string"}



### **Step 2: Download the audio, split it (if selected) and launch RVC WebUI**

#### **2.1. Download Youtube Audio and Split it**

In [ ]:
#@title - Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title - Install libraries for Youtube WAV download
if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  !pip install yt_dlp
  !pip install ffmpeg
  !mkdir youtubeaudio

In [ ]:
#@title - Download audio
from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)

In [ ]:
#@title - Install Demucs for separating audio
!python3 -m pip install -U demucs

In [ ]:
#@title - Separate vocal and instrument/noise using Demucs
import subprocess
AUDIO_INPUT = f"/content/youtubeaudio/{AUDIO_NAME}.wav"

if dataset == "Drive":
  command = f"demucs --two-stems=vocals {drive_path}"
elif dataset == "Youtube":
  command = f"demucs --two-stems=vocals {AUDIO_INPUT}"

result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())

In [ ]:
#@title - Copy downloaded audio to Google Drive
!mkdir -p /content/drive/MyDrive/audio/{AUDIO_NAME}
!cp -r /content/separated/htdemucs/{AUDIO_NAME}/* /content/drive/MyDrive/audio/{AUDIO_NAME}
if dataset == "Youtube":
  !cp -r /content/youtubeaudio/{AUDIO_NAME}.wav /content/drive/MyDrive/audio/{AUDIO_NAME}


In [ ]:
#@title - Install library for audio splitting
if AudioMode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif AudioMode ==  "Splitting":
  !pip install numpy
  !pip install librosa
  !pip install soundfile
  !mkdir -p dataset/{AUDIO_NAME}

In [ ]:
#@title - Split audio to smaller duration
import numpy as np
import librosa
import soundfile


# This function is obtained from librosa.
def get_rms(
    y,
    *,
    frame_length=2048,
    hop_length=512,
    pad_mode="constant",
):
    padding = (int(frame_length // 2), int(frame_length // 2))
    y = np.pad(y, padding, mode=pad_mode)

    axis = -1
    # put our new within-frame axis at the end for now
    out_strides = y.strides + tuple([y.strides[axis]])
    # Reduce the shape on the framing axis
    x_shape_trimmed = list(y.shape)
    x_shape_trimmed[axis] -= frame_length - 1
    out_shape = tuple(x_shape_trimmed) + tuple([frame_length])
    xw = np.lib.stride_tricks.as_strided(
        y, shape=out_shape, strides=out_strides
    )
    if axis < 0:
        target_axis = axis - 1
    else:
        target_axis = axis + 1
    xw = np.moveaxis(xw, -1, target_axis)
    # Downsample along the target axis
    slices = [slice(None)] * xw.ndim
    slices[axis] = slice(0, None, hop_length)
    x = xw[tuple(slices)]

    # Calculate power
    power = np.mean(np.abs(x) ** 2, axis=-2, keepdims=True)

    return np.sqrt(power)


class Slicer:
    def __init__(self,
                 sr: int,
                 threshold: float = -40.,
                 min_length: int = 5000,
                 min_interval: int = 300,
                 hop_size: int = 20,
                 max_sil_kept: int = 5000):
        if not min_length >= min_interval >= hop_size:
            raise ValueError('The following condition must be satisfied: min_length >= min_interval >= hop_size')
        if not max_sil_kept >= hop_size:
            raise ValueError('The following condition must be satisfied: max_sil_kept >= hop_size')
        min_interval = sr * min_interval / 1000
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.win_size = min(round(min_interval), 4 * self.hop_size)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def _apply_slice(self, waveform, begin, end):
        if len(waveform.shape) > 1:
            return waveform[:, begin * self.hop_size: min(waveform.shape[1], end * self.hop_size)]
        else:
            return waveform[begin * self.hop_size: min(waveform.shape[0], end * self.hop_size)]

    def slice(self, waveform):
        if len(waveform.shape) > 1:
            samples = waveform.mean(axis=0)
        else:
            samples = waveform
        if samples.shape[0] <= self.min_length:
            return [waveform]
        rms_list = get_rms(y=samples, frame_length=self.win_size, hop_length=self.hop_size).squeeze(0)
        sil_tags = []
        silence_start = None
        clip_start = 0
        for i, rms in enumerate(rms_list):
            # Keep looping while frame is silent.
            if rms < self.threshold:
                # Record start of silent frames.
                if silence_start is None:
                    silence_start = i
                continue
            # Keep looping while frame is not silent and silence start has not been recorded.
            if silence_start is None:
                continue
            # Clear recorded silence start if interval is not enough or clip is too short
            is_leading_silence = silence_start == 0 and i > self.max_sil_kept
            need_slice_middle = i - silence_start >= self.min_interval and i - clip_start >= self.min_length
            if not is_leading_silence and not need_slice_middle:
                silence_start = None
                continue
            # Need slicing. Record the range of silent frames to be removed.
            if i - silence_start <= self.max_sil_kept:
                pos = rms_list[silence_start: i + 1].argmin() + silence_start
                if silence_start == 0:
                    sil_tags.append((0, pos))
                else:
                    sil_tags.append((pos, pos))
                clip_start = pos
            elif i - silence_start <= self.max_sil_kept * 2:
                pos = rms_list[i - self.max_sil_kept: silence_start + self.max_sil_kept + 1].argmin()
                pos += i - self.max_sil_kept
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                    clip_start = pos_r
                else:
                    sil_tags.append((min(pos_l, pos), max(pos_r, pos)))
                    clip_start = max(pos_r, pos)
            else:
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                else:
                    sil_tags.append((pos_l, pos_r))
                clip_start = pos_r
            silence_start = None
        # Deal with trailing silence.
        total_frames = rms_list.shape[0]
        if silence_start is not None and total_frames - silence_start >= self.min_interval:
            silence_end = min(total_frames, silence_start + self.max_sil_kept)
            pos = rms_list[silence_start: silence_end + 1].argmin() + silence_start
            sil_tags.append((pos, total_frames + 1))
        # Apply and return slices.
        if len(sil_tags) == 0:
            return [waveform]
        else:
            chunks = []
            if sil_tags[0][0] > 0:
                chunks.append(self._apply_slice(waveform, 0, sil_tags[0][0]))
            for i in range(len(sil_tags) - 1):
                chunks.append(self._apply_slice(waveform, sil_tags[i][1], sil_tags[i + 1][0]))
            if sil_tags[-1][1] < total_frames:
                chunks.append(self._apply_slice(waveform, sil_tags[-1][1], total_frames))
            return chunks

if AudioMode == "Separate":
    print("Mode is set to Separate. Skipping this section")

elif AudioMode ==  "Splitting":
  audio, sr = librosa.load(f'/content/separated/htdemucs/{AUDIO_NAME}/vocals.wav', sr=None, mono=False)  # Load an audio file with librosa.
  slicer = Slicer(
      sr=sr,
      threshold=-40,
      min_length=5000,
      min_interval=200,
      hop_size=10,
      max_sil_kept=500
  )
  chunks = slicer.slice(audio)
  for i, chunk in enumerate(chunks):
      if len(chunk.shape) > 1:
          chunk = chunk.T  # Swap axes if the audio is stereo.
      soundfile.write(f'/content/dataset/{AUDIO_NAME}/split_{i}.wav', chunk, sr)  # Save sliced audio files with soundfile.

In [ ]:
#@title - Copy the splited audio to Google Drive
if AudioMode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif AudioMode ==  "Splitting":
  !mkdir -p /content/drive/MyDrive/dataset/{AUDIO_NAME}
  !cp -r /content/dataset/{AUDIO_NAME}/* /content/drive/MyDrive/dataset/{AUDIO_NAME}

#### **2.2. Firing up RVC Training V2**

In [ ]:
#@title - Cloning Github

!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!cp -r /content/drive/MyDrive/RVC/weights/* /content/Retrieval-based-Voice-Conversion-WebUI/assets/weights
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!cp -r /content/drive/MyDrive/RVC/{MODELNAME}/* /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
%cd /content/Retrieval-based-Voice-Conversion-WebUI

In [ ]:
#@title - Install library for RVC
!pip install pyworld==0.3.2
!pip install -r requirements.txt

In [ ]:
#@title - Install aria2
!apt -y install -qq aria2

In [ ]:
#@title - Download Pretrained Model for Training
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained -o f0G48k.pth

#RVC V2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/pretrained_v2 -o f0G48k.pth

In [ ]:
#@title Download Pretrained Model for Audio Splitting
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/5_HP-Karaoke-UVR.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o 5_HP-Karaoke-UVR.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP2_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP3_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP3_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5_only_main_vocal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o HP5_only_main_vocal.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoAggressive.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o VR-DeEchoAggressive.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoDeReverb.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o VR-DeEchoDeReverb.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoNormal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights -o VR-DeEchoNormal.pth


In [ ]:
#@title - Download hubert_base and rmvpe
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI/assets -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -d /content/Retrieval-based-Voice-Conversion-WebUI/assets -o rmvpe.pt

In [ ]:
#@title - Run Web
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3

In [ ]:
#@title - Copy Training Result to Drive
if RVCMode == "training":
  !mkdir -p /content/drive/MyDrive/RVC/{MODELNAME}
  !mkdir -p /content/drive/MyDrive/RVC/weights

  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_*.pth /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_*.pth /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/trained*.index /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/RVC/{MODELNAME}

  !cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/RVC/weights/{MODELNAME}.pth
elif RVCMode == "inference":
  print("Mode set to inference. Skipping this section")